In [1]:
import pandas as pd
import numpy as np
#!pip3 install mlxtend

from sklearn import model_selection

from sklearn import tree #C4.5
from sklearn.ensemble import BaggingClassifier #Bagging
from sklearn.ensemble import AdaBoostClassifier #Boosting
from sklearn.ensemble import RandomForestClassifier #Random Forests
from sklearn.ensemble import GradientBoostingClassifier #XGBoost
from mlxtend.classifier import StackingClassifier #Stacking

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
import time
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.options.display.max_colwidth = 150

data_glass = pd.read_csv("glass.data")

# to store the models and their results (to report)
models, results, tempos = list(), list(), list()

print('Glass:', np.shape(data_glass))

Glass: (214, 11)


In [2]:
data_glass = shuffle(data_glass)
data_glass.head()

,a,b,c,d,e,f,g,h,i,j,target
61,62,1.51977,13.81,3.58,1.32,71.72,0.12,8.67,0.69,0.00,1
44,45,1.51786,12.73,3.43,1.19,72.95,0.62,8.76,0.00,0.30,1
200,201,1.51508,15.15,0.00,2.25,73.50,0.00,8.34,0.63,0.00,7
33,34,1.51753,12.57,3.47,1.38,73.39,0.60,8.55,0.00,0.06,1
203,204,1.51658,14.80,0.00,1.99,73.11,0.00,8.28,1.71,0.00,7


In [3]:
y = data_glass['target']
data_glass = data_glass.drop(columns = ['target', 'a'])

In [4]:
# Normalizing the Glass dataset: standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
scaler.fit(data_glass)
data_glass[:] = scaler.transform(data_glass)
data_glass.head()

,b,c,d,e,f,g,h,i,j
61,0.463594,0.493621,0.622270,-0.250765,-1.204728,-0.579492,-0.202111,1.038095,-0.586451
44,-0.166819,-0.832031,0.518033,-0.511756,0.387022,0.188951,-0.138723,-0.352877,2.499627
200,-1.084383,2.138411,-1.865511,1.616323,1.098781,-0.763919,-0.434534,0.917141,-0.586451
33,-0.275738,-1.028424,0.545830,-0.130308,0.956429,0.158213,-0.286629,-0.352877,0.030765
203,-0.589294,1.708802,-1.865511,1.094342,0.594079,-0.763919,-0.476793,3.094313,-0.586451


In [15]:
C45 = tree.DecisionTreeClassifier() 

clf1 = KNeighborsClassifier(n_neighbors=1)
#clf2 = GaussianNB()
lr = LogisticRegression()

all_models=[C45, 
            BaggingClassifier(C45, max_samples=0.4),
            AdaBoostClassifier(C45, n_estimators=100),
            RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0),
            GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, subsample=1.0),
            StackingClassifier(classifiers=[clf1, clf2, C45], meta_classifier=lr)]

models.extend(all_models)

In [16]:
kf = KFold(n_splits=10, shuffle=True, random_state=2019)    

Running models for Glass dataset:

In [17]:
#For each model:
aux = 0
for model in all_models:
    print('\n\n======= Model: ', model, '=======')
    # for each fold:
    accuracies = list()
    times = list()
    
    for i, (train_index, test_index) in enumerate(kf.split(data_glass)):
        #Builds the train and validation dataset, according to the current fold:
        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
        X_train, X_valid = data_glass.iloc[train_index,:].copy(), data_glass.iloc[test_index,:].copy()
        # print("Fold", i)
        
        #This condition is for the Stacking ensemble
        if (aux == 5):
                        
            for clf, label in zip([clf1, clf2, C45, model], ['KNN', 'Naive Bayes','C4.5','StackingClassifier']):

                scores = model_selection.cross_val_score(clf, data_glass, y, cv=10, scoring='accuracy')
                print("Mean accuracy: ", scores.mean(), label)
                
            break
                
        else:
            start = time.time()
            model.fit(X_train, y_train)
            end = time.time()
            times.append(end - start)
            pred = model.predict(X_valid)
            # print(confusion_matrix(y_valid, pred, labels=[1,2,3,5,6,7]))
            acc = accuracy_score(pred, y_valid)
            accuracies.append(acc)
            
    if(aux != 5):        
        print('====\nFinal results: \nMean accuracy:', np.mean(accuracies))
        print('Mean traning model time: ', np.mean(times))
        results.append(np.mean(accuracies))
        tempos.append(np.mean(times))
    
    aux += 1



======= Model:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best') =======
====
Final results: 
Mean accuracy: 0.6774891774891776
Mean traning model time:  0.0028728485107421876


======= Model:  BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrea

/home/afonso/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, w

Mean accuracy:  0.6196605809649289 StackingClassifier


/home/afonso/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/afonso/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: 